# Poly Prep
Cascade Tuholske Dec 2020


In [54]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
from rasterio import features
import rasterio
import rasterio.mask
import rioxarray as rio
from skimage.filters import rank
from rasterio.enums import Resampling
from whitebox import WhiteboxTools

In [2]:
# file paths 
data_in = '/Users/cascade/Github/PopGridCompare/data/raw/'
data_out = '/Users/cascade/Github/PopGridCompare/data/interim/'

## Dissolve polygons for Mozambique, Zimbabwe & Malawai 

In [ ]:
# polys in
zwe = gpd.read_file(data_in+'GDAM/gadm36_ZWE_shp/gadm36_ZWE_0.shp')
mwi = gpd.read_file(data_in+'GDAM/gadm36_MWI_shp/gadm36_MWI_0.shp')
moz = gpd.read_file(data_in+'GDAM/gadm36_MOZ_shp/gadm36_MOZ_0.shp')

In [ ]:
dfs = [zwe, mwi, moz]
polys = gpd.GeoDataFrame(pd.concat(dfs))

In [ ]:
polys.plot()
polys.head()

In [ ]:
# Dissolve 
polys['ID'] = 'M-M-Z'
diss = polys.dissolve(by = 'ID')

In [ ]:
diss.plot()

In [ ]:
# write
diss.to_file(data_out+'M-M-Z.shp')

## Drop Islands for Ecuador

In [ ]:
# polys in
ecu = gpd.read_file(data_in+'GDAM/gadm36_ECU_shp/gadm36_ECU_0.shp')
flood = gpd.read_file(data_in+'ECU-Floods/Susceptibility/FF_suscept_PCA.shp')
fn_out = data_out+'ECU-clip.shp'

In [ ]:
# change crs
flood = flood.to_crs(ecu.crs)

In [ ]:
# dissolve floods and get bounding box
flood['ID'] = 'ECU'
diss = flood.dissolve(by = 'ID')
diss.plot()
bb = box(*diss.total_bounds)

In [ ]:
# make bb a geodata frame
df = pd.DataFrame(['ECU'])
df['geometry'] = bb
bb_gdf = gpd.GeoDataFrame(df).buffer(1)

In [ ]:
# plot them

axs = bb_gdf.plot(color = 'green', alpha = 0.7, figsize = (18, 18))
ecu.plot(ax = axs, figsize = (8, 8))

In [ ]:
bb_gdf.crs

In [ ]:
# clip to buffer (will throw warning; that's ok)
clip = gpd.clip(ecu, bb_gdf)
clip.plot()

In [ ]:
clip.to_file(fn_out)

## Turn GDAM polygons into raster bianaries

In [ ]:
def poly_to_raster (rst, polys, value, touched, out_fn, fill_value):
    """Function makes a raster from a list of polygons
    
    Args:   rst = input raster already read in as a rasterio object to act as a template
            polys = input polygons already read in as a gpd dataframe
            value = col with value to burn into raster
            touched = bool, if True all pixels touched (not centers) are burned into raster
            out_fn = out file name 
            fill_value = value to revalue input raster before burning in polygons 
    
    """

    meta = rst.meta.copy() # copy meta data from rst
    out_arr = rst.read(1) # get an array to burn shapes
    out_arr.fill(fill_value) # revalue rst to an Nan Value before burning in polygons
    
    # extract geom and values to burn
    shapes = ((geom,value) for geom, value in zip(polys['geometry'], polys[value])) 
    
    # burn shapes intp an array
    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=rst.transform, all_touched=touched)
    
    # write our raster to disk
    with rasterio.open(out_fn, 'w', **meta) as out:
        out.write_band(1, burned)

In [ ]:
def raster_clip(rst_in, polys, rst_out):
    
    """ function clips a raster and saves it out
    args:
        rst_in = raster you want to clip
        polys = polys you want to clip to
        rst_out = name of clipped raster
    """
    
    # clip raster
    with rasterio.open(rst_fn) as src:
        out_image, out_transform = rasterio.mask.mask(src, polys, crop=True, nodata = 0)
        out_meta = src.meta
        
    # Update meta data
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    # write image 
    with rasterio.open(fn_out, "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
# polygons to burn
# GDAM/gadm36_NPL_shp
polys = gpd.read_file(data_in+'GDAM/gadm36_NPL_shp/gadm36_NPL_0.shp')

# update columns
polys = polys[['geometry']]
polys['value'] = 1

fn_out = data_out+'Nepal-rst.tif'

In [ ]:
rst = rasterio.open(data_in+'gpw_v4/gpw-v4-population-count-rev11_2015_30_sec_tif/gpw_v4_population_count_rev11_2015_30_sec.tif')
value = 'value'
touched = False # see rasterio documentation, using un-touched in this analysis 
fill_value = 0

In [ ]:
poly_to_raster(rst, polys, value, touched, fn_out, fill_value)

In [ ]:
# Now clip the raster
rst_fn = fn_out
rst_out = fn_out
polys = polys['geometry']

raster_clip(rst_fn, polys, rst_out)

## Idai Flood Raster
Data from human data exchange, at 30m resolution values = 1 & 255 <br>
Need to resample with a majority filter at 1-km

In [47]:
def resample(fn_in, fn_out, scale_factor, method):
    
    """ Resamples a raster and save it out
    Args:
        fn_in = file path and name of tif input as str
        fn_out = file path and name of tif output as str 
        scale_factor = factor to up or down scale a pixel as float
        method = method to resample (rasterio object), see rasterio documentation
    """
    
    with rasterio.open(fn_in) as dataset:

        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor),
                int(dataset.width * scale_factor)
            ),
            resampling=method
        )

        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )
    
    # meta data to write out
    out_meta = dataset.meta

    # Update meta data
    out_meta.update({"driver": "GTiff",
             "height": data.shape[1],
             "width": data.shape[2],
             "transform": transform})

    # write image 
    with rasterio.open(fn_out, "w", **out_meta) as dest:
        dest.write(data)

In [48]:
# Resample
rst_fn = data_in+'Idai/HDX/moz_totalfloodextent/MOZ_TotalFloodExtent12to2.tif'
fn_out = data_out+'Idai_flood1km.tif'

In [49]:
# resample(fn_in, fn_out, 1/10, Resampling.mode) ... 
# flood pixels are 90m so 1/10 will make 1-km
resample(rst_fn, fn_out, 1/11, Resampling.mode)

In [78]:
# Reproject all datasets to GPWv4

# files
rst_fn = data_out+'Idai_flood1km.tif'
mmz_fn = data_out+'M-M-Z-rst.tif'
rst_out = data_out+'Idai_flood1km-matched.tif'

# open & reproject
rst = rio.open_rasterio(rst_fn)
mmz = rio.open_rasterio(mmz_fn)
match = rst.rio.reproject_match(mmz)

# fix fill value
match.data = np.where(match.data == 255, 0, match.data)
match.data = match.data.astype('int8')
match.attrs['_FillValue'] = -99
match.rio.to_raster(rst_out)

In [69]:
# re-set values
rst_fn = data_in+'Idai/HDX/moz_totalfloodextent/MOZ_TotalFloodExtent12to2.tif'
fn_out = data_out+'Idai_90m_zeros.tif'
rst = rasterio.open(rst_fn)
arr = rst.read(1)
arr_out = np.where(arr == 255, 0, arr)

out_metta = rst.meta
with rasterio.open(fn_out, "w", **out_meta) as dest:
    dest.write(arr_out, 1)

print(rasterio.open(fn_out).meta)
print(np.unique(rasterio.open(fn_out).read(1)))

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 255.0, 'width': 861, 'height': 775, 'count': 1, 'crs': CRS.from_epsg(3857), 'transform': Affine(951.3579262455364, 0.0, 3500648.1262464076,
       0.0, -952.2357028390246, -1663877.0266072527)}
[0 1]


In [70]:
# Majority filter 

rst_fn = data_out+'Idai_90m_zeros.tif'
fn_out = data_out+'Idai_flood-majority.tif'

wbt = WhiteboxTools()
wbt.majority_filter(rst_fn, fn_out, filterx=11, filtery=11)

./whitebox_tools --run="MajorityFilter" --input='/Users/cascade/Github/PopGridCompare/data/interim/Idai_90m_zeros.tif' --output='/Users/cascade/Github/PopGridCompare/data/interim/Idai_flood-majority.tif' --filterx=11 --filtery=11 -v

*****************************
* Welcome to MajorityFilter *
*****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progress: 42%
Progress: 43%
Progress: 44%
Progress: 45%
Progress: 46%
Progress: 47

0